# Corpus

In [679]:
import pandas as pd                                     
import numpy as np                                      
import os 



import matplotlib.pyplot as plt

from datetime import datetime

%matplotlib inline
import seaborn as sns                                   # For pretty plots
import gensim
import nltk


from os import path
from wordcloud import WordCloud
from wordcloud import STOPWORDS
from PIL import Image
from gensim import corpora, models, similarities
from nltk.corpus import stopwords

In [680]:
emails = pd.read_csv("hillary-clinton-emails/Emails.csv")
#emails.head()

In [681]:
emails_text = emails['ExtractedBodyText'].dropna()
#emails_text = emails_text.head(100)
#emails_text.shape
type(emails_text)

pandas.core.series.Series

In [682]:
emails_text = emails_text.apply(lambda x: x.lower())

In [683]:
emails_text = emails_text.apply(lambda x: x\
                                .replace(':',' ')\
                                .replace('—',' ')\
                                .replace('-',' ')\
                                .replace('.',' ')\
                                .replace(',',' ')\
                                .replace('.',' ')\
                                .replace('<',' ')\
                                .replace('>',' ')\
                                .replace('=',' ')\
                                .replace('•',' ')\
                                .replace("\\",' ')\
                                .replace('\n', ' ')\
                                .replace('^',' ')\
                                .replace('\\',' ')\
                                .replace('?',' ')\
                                .replace('\'',' ')\
                               )
#emails_text = emails_text.apply(lambda x: x.replace('—',' '))


In [684]:
type(emails_text)
#emails_text.iloc[0]
#emails_text

pandas.core.series.Series

In [685]:
#emails_text.head

In [686]:
documents = emails_text.tolist()
#documents

In [687]:
type(documents)
#documents

list

In [688]:
len(documents)

6742

In [689]:
stoplist = set(STOPWORDS)
stoplist.add('pls')
stoplist.add('yes')
stoplist.add('call')
stoplist.add('pm')
stoplist.add('no')
stoplist.add('com')
stoplist.add('doc')
stoplist.add('docx')
stoplist.add('pdf')
stoplist.add('mr')
stoplist.add('mrs')
stoplist.add('call')
stoplist.add('need')
stoplist.add('one')
stoplist.add('two')
stoplist.add('fyi')
stoplist.add('00')
stoplist.add('will')
stoplist.add('know')
stoplist.add('re')
stoplist.add('ok')
stoplist.add('also')
stoplist.add('see')
stoplist.add('us')
stoplist.add('good')
stoplist.add('thx')
stoplist.add('new')
stoplist.add('go')
stoplist.add('you')
stoplist.add('now')
stoplist.add('done')


for i in range(10,100) :
    stoplist.add(i)
print(stoplist)

{'are', 'itself', '', "she'll", 'now', 'two', 10, 11, 'most', "couldn't", 'an', 'after', 'its', 'been', 'ours', 'yourself', 'done', 12, 13, 14, 15, 16, "don't", "you're", 'in', 20, 21, 22, 23, 24, 25, 26, 27, 28, 'would', 30, "he'll", 32, 33, 34, 35, 36, 37, 38, 39, 40, 'there', 'ok', 'thx', "she's", 45, 46, 47, 48, 49, 50, "he's", 'herself', "they'd", 'was', 55, 56, 57, "you'd", 59, 'docx', 61, 62, 63, 'both', 'could', 'no', 67, "mustn't", 69, 70, 71, 72, 73, 74, 'i', 'did', 'see', 'over', "aren't", "shouldn't", 'know', 17, 18, 84, 'doc', 'one', 19, 88, 89, 'ourselves', 91, 'such', 93, 94, 95, 96, "haven't", 98, 99, 'is', 'this', "shan't", "we're", "he'd", 'http', 'before', "it's", 'nor', 'our', 'very', 'theirs', 'pm', "that's", 'does', 'any', 'get', 29, "can't", 'doing', 'the', "you've", "we'd", 'and', "hasn't", 'hers', 'so', 31, "why's", 'good', 'have', 'off', 'them', 'having', 'again', 'also', "i'd", 'yours', 're', "we'll", 'how', 'cannot', 'during', "you'll", 'yourselves', 'new', 

In [690]:
# remove list contains on stoplist and tokenize
texts = [[word for word in document.lower().split() if word not in stoplist and len(word) > 1]
         for document in documents]

In [691]:
len(texts)

6742

In [692]:
# counts the number of appartion of a word
from collections import defaultdict
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1

In [693]:
len(frequency)

34509

In [694]:
# remove word that apprears only once
texts = [[token for token in text if frequency[token] > 1]
      for text in texts]

In [695]:
len(texts)

6742

In [696]:
#texts

In [697]:
dictionary = corpora.Dictionary(texts)
dictionary.save("part3/deerwester-email.dict")

In [698]:
print(dictionary)

Dictionary(17670 unique tokens: ['145', 'pouch', 'capabilities', 'rooms', 'mirage']...)


In [699]:
corpus = [dictionary.doc2bow(text) for text in texts]

In [700]:
#print(corpus)

In [701]:
corpora.MmCorpus.serialize("part3/deerwester-email.mm", corpus)

In [702]:
dictionary = corpora.Dictionary.load("part3/deerwester-email.dict")


In [703]:
model = models.LdaModel(corpus, id2word=dictionary, num_topics=50)

In [704]:
model.print_topics(5)

[(21,
  '0.026*"palestinian" + 0.021*"negotiating" + 0.011*"gaza" + 0.011*"arabs" + 0.010*"institution" + 0.010*"washington" + 0.010*"strobe" + 0.009*"walter" + 0.008*"brookings" + 0.008*"boy"'),
 (41,
  '0.006*"people" + 0.005*"government" + 0.005*"state" + 0.005*"president" + 0.004*"military" + 0.004*"obama" + 0.004*"time" + 0.004*"think" + 0.003*"health" + 0.003*"security"'),
 (28,
  '0.013*"labour" + 0.010*"david" + 0.008*"party" + 0.007*"ed" + 0.007*"obama" + 0.006*"policy" + 0.006*"iran" + 0.006*"parties" + 0.005*"president" + 0.005*"tax"'),
 (37,
  '0.018*"marie" + 0.016*"anne" + 0.014*"books" + 0.013*"fco" + 0.012*"activists" + 0.012*"slaughter" + 0.012*"website" + 0.012*"calling" + 0.009*"weekend" + 0.009*"camera"'),
 (34,
  '0.008*"israel" + 0.007*"state" + 0.007*"arab" + 0.007*"government" + 0.006*"port" + 0.006*"january" + 0.005*"military" + 0.005*"au" + 0.005*"united" + 0.004*"leaders"')]

In [705]:
print(model)

LdaModel(num_terms=17670, num_topics=50, decay=0.5, chunksize=2000)


In [706]:
#print(corpus)